In [45]:
import random
import csv
import time
import math
from collections import defaultdict
import os
import pandas as pd
import matplotlib.pyplot as plt

def seedseed(C, n, v1, v2):
    return v1 * C + v2 * (C / 10) + n * (C / 100) + C

def bppgen(N, C, V1, V2, seed):
    if not (0 < V1 <= V2 <= 1):
        print("Input parameters out of range, aborting.")
        return None

    random.seed(seed)

    weights = []
    for _ in range(N):
        rand_num = random.uniform(0, 1)
        weight = int(V1 * C + (V2 - V1) * rand_num * C + rand_num)
        weights.append(weight)

    unique_weights = {}
    for weight in weights:
        if weight in unique_weights:
            unique_weights[weight] += 1
        else:
            unique_weights[weight] = 1

    return unique_weights

# Set the parameters for different cases
parameters = [
    (100, 1000, 0.01, 0.05),
    (100, 1000, 0.7, 0.9),
    (100, 1000, 0.4, 0.5),
    (100, 1000, 0.1, 0.9),
    (100, 1000, 0.05, 0.6),
    (100, 1000, 0.25, 0.5),
    (200, 1000, 0.01, 0.05),
    (200, 1000, 0.7, 0.9),
    (200, 1000, 0.4, 0.5),
    (200, 1000, 0.1, 0.9),
    (200, 1000, 0.05, 0.6),
    (200, 1000, 0.25, 0.5),
]

# Create output directory if not exists
output_dir = 'bppgen_output'
os.makedirs(output_dir, exist_ok=True)

for N, C, V1, V2 in parameters:

    filename = f'N={N}_C={C}_V1={V1}_V2={V2}.csv'
    filepath = os.path.join(output_dir, filename)
    
    with open(filepath, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Example", "Weight", "Quantity"])

        for example_num in range(1, 1001):
            seed_value = seedseed(C, N, V1, V2) 
            unique_weights = bppgen(N, C, V1, V2, seed_value + example_num - 1)
            if unique_weights:
                for weight, quantity in unique_weights.items():
                    writer.writerow([example_num, weight, quantity])

    print(f"Data has been written to {filepath}")

Data has been written to bppgen_output\N=100_C=1000_V1=0.01_V2=0.05.csv
Data has been written to bppgen_output\N=100_C=1000_V1=0.7_V2=0.9.csv
Data has been written to bppgen_output\N=100_C=1000_V1=0.4_V2=0.5.csv
Data has been written to bppgen_output\N=100_C=1000_V1=0.1_V2=0.9.csv
Data has been written to bppgen_output\N=100_C=1000_V1=0.05_V2=0.6.csv
Data has been written to bppgen_output\N=100_C=1000_V1=0.25_V2=0.5.csv
Data has been written to bppgen_output\N=200_C=1000_V1=0.01_V2=0.05.csv
Data has been written to bppgen_output\N=200_C=1000_V1=0.7_V2=0.9.csv
Data has been written to bppgen_output\N=200_C=1000_V1=0.4_V2=0.5.csv
Data has been written to bppgen_output\N=200_C=1000_V1=0.1_V2=0.9.csv
Data has been written to bppgen_output\N=200_C=1000_V1=0.05_V2=0.6.csv
Data has been written to bppgen_output\N=200_C=1000_V1=0.25_V2=0.5.csv


In [46]:

def read_csv(file_path):
    examples = defaultdict(list)
    with open(file_path, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            example_num = int(row["Example"])
            weight = int(row["Weight"])
            quantity = int(row["Quantity"])
            examples[example_num].extend([weight] * quantity)
    return examples

def calculate_lb(weights, bin_capacity):
    total_weight = sum(weights)
    lb = math.ceil(total_weight / bin_capacity)
    return lb

def ff(weights, bin_capacity):
    bins = []

    for weight in weights:
        placed = False
        for i in range(len(bins)):
            if bins[i] + weight <= bin_capacity:
                bins[i] += weight
                placed = True
                break
        if not placed:
            bins.append(weight)
    
    return len(bins)


def ffd(weights, bin_capacity):
    weights.sort(reverse=True) 
    bins = []

    for weight in weights:
        placed = False
        for i in range(len(bins)):
            if bins[i] + weight <= bin_capacity:
                bins[i] += weight
                placed = True
                break
        if not placed:
            bins.append(weight)
    
    return len(bins)


def bf(weights, bin_capacity):
    bins = []

    for weight in weights:
        best_fit_index = -1
        min_space_left = bin_capacity + 1

        for i in range(len(bins)):
            space_left = bin_capacity - bins[i]
            if space_left >= weight and space_left < min_space_left:
                best_fit_index = i
                min_space_left = space_left

        if best_fit_index == -1:
            bins.append(weight)
        else:
            bins[best_fit_index] += weight
    
    return len(bins)


def bfd(weights, bin_capacity):
    weights.sort(reverse=True)  
    bins = []

    for weight in weights:
        best_fit_index = -1
        min_space_left = bin_capacity + 1

        for i in range(len(bins)):
            space_left = bin_capacity - bins[i]
            if space_left >= weight and space_left < min_space_left:
                best_fit_index = i
                min_space_left = space_left

        if best_fit_index == -1:
            bins.append(weight)
        else:
            bins[best_fit_index] += weight
    
    return len(bins)


def wf(weights, bin_capacity):
    bins = []

    for weight in weights:
        worst_fit_index = -1
        max_space_left = -1

        for i in range(len(bins)):
            space_left = bin_capacity - bins[i]
            if space_left >= weight and space_left > max_space_left:
                worst_fit_index = i
                max_space_left = space_left

        if worst_fit_index == -1:
            bins.append(weight)
        else:
            bins[worst_fit_index] += weight
    
    return len(bins)


def wfd(weights, bin_capacity):
    weights.sort(reverse=True)  
    bins = []

    for weight in weights:
        worst_fit_index = -1
        max_space_left = -1

        for i in range(len(bins)):
            space_left = bin_capacity - bins[i]
            if space_left >= weight and space_left > max_space_left:
                worst_fit_index = i
                max_space_left = space_left

        if worst_fit_index == -1:
            bins.append(weight)
        else:
            bins[worst_fit_index] += weight
    
    return len(bins)



def multi_bin_ff(weights, bin_capacity, num_bins):
    bins = [0] * num_bins

    for weight in weights:
        placed = False
        for i in range(len(bins)):
            if bins[i] + weight <= bin_capacity:
                bins[i] += weight
                placed = True
                break
        
        if not placed:
            return False  # Place all items in a specified number of bins

    return True  

def multi_bin_ffd(weights, bin_capacity, num_bins):
    weights.sort(reverse=True)  
    bins = [0] * num_bins

    for weight in weights:
        placed = False
        for i in range(len(bins)):
            if bins[i] + weight <= bin_capacity:
                bins[i] += weight
                placed = True
                break
        
        if not placed:
            return False 

    return True  

def multi_bin_bf(weights, bin_capacity, num_bins):
    bins = [0] * num_bins

    for weight in weights:
        best_fit_index = -1
        min_space_left = bin_capacity + 1

        for i in range(len(bins)):
            space_left = bin_capacity - bins[i]
            if space_left >= weight and space_left < min_space_left:
                best_fit_index = i
                min_space_left = space_left

        if best_fit_index == -1:
            return False  
        else:
            bins[best_fit_index] += weight

    return True  

def multi_bin_bfd(weights, bin_capacity, num_bins):
    weights.sort(reverse=True)  
    bins = [0] * num_bins

    for weight in weights:
        best_fit_index = -1
        min_space_left = bin_capacity + 1

        for i in range(len(bins)):
            space_left = bin_capacity - bins[i]
            if space_left >= weight and space_left < min_space_left:
                best_fit_index = i
                min_space_left = space_left

        if best_fit_index == -1:
            return False  
        else:
            bins[best_fit_index] += weight

    return True  
    
def multi_bin_wf(weights, bin_capacity, num_bins):
    bins = [0] * num_bins

    for weight in weights:
        worst_fit_index = -1
        max_space_left = -1

        for i in range(len(bins)):
            space_left = bin_capacity - bins[i]
            if space_left >= weight and space_left > max_space_left:
                worst_fit_index = i
                max_space_left = space_left

        if worst_fit_index == -1:
            return False  
        else:
            bins[worst_fit_index] += weight

    return True  
    

def multi_bin_wfd(weights, bin_capacity, num_bins):
    weights.sort(reverse=True)  
    bins = [0] * num_bins

    for weight in weights:
        worst_fit_index = -1
        max_space_left = -1

        for i in range(len(bins)):
            space_left = bin_capacity - bins[i]
            if space_left >= weight and space_left > max_space_left:
                worst_fit_index = i
                max_space_left = space_left

        if worst_fit_index == -1:
            return False  
        else:
            bins[worst_fit_index] += weight

    return True  

 
def find_min_bins(weights, bin_capacity, algorithm, lb,algorithm2):
    left = lb
    right = algorithm2(weights, bin_capacity)
    while left < right:
        mid = (left + right) // 2
        if algorithm(weights, bin_capacity, mid):
            right = mid
        else:
            left = mid + 1

    return left   




In [47]:
import os
import csv
import time
from collections import defaultdict
import matplotlib.pyplot as plt

output_dir = 'bppgen_output'
bin_capacity = 1000  


csv_files = [f for f in os.listdir(output_dir) if f.endswith('.csv')]


summary_file = os.path.join(output_dir, 'summary_results.csv')




with open(summary_file, mode='w', newline='') as summary_csv:
    summary_writer = csv.writer(summary_csv)
    summary_writer.writerow([
        "CSV File", "Algorithm", "Avg  computation time (s)", "Avg Deviation from LB"
    ])
    

    for csv_file in csv_files:
        file_path = os.path.join(output_dir, csv_file)
        examples = read_csv(file_path)

        print(f"processing： {csv_file}")

        num_runs = 100
        avg_results = defaultdict(lambda: {'total_time': 0, 'total_deviation': 0, 'count': 0})

        for example_num, weights in examples.items():
            lb = calculate_lb(weights, bin_capacity)
            min_bins_dict = {}
        
            
            for algo_name, algo_func in [('FF', ff), ('BF', bf), ('WF', wf), 
                                         ('FFD', ffd), ('BFD', bfd), ('WFD', wfd)]:
                start_time = time.perf_counter()
                for _ in range(num_runs):
                    min_bins = algo_func(weights[:], bin_capacity)
                run_time = (time.perf_counter() - start_time) / num_runs
                deviation = min_bins - lb
                avg_results[algo_name]['total_time'] += run_time
                avg_results[algo_name]['total_deviation'] += deviation
                avg_results[algo_name]['count'] += 1
                min_bins_dict[algo_name] = {'min_bins': min_bins, 'run_time': run_time}      

                with open('bppgen_output\detail_results.csv', 'a', newline='') as csvfile:
                    csvwriter = csv.writer(csvfile)
                
                    
                    if csvfile.tell() == 0:
                        csvwriter.writerow(['File Name', 'Example Number', 'Algorithm', 'Min Bins', 'LB', 'Computation Time (s)'])
                    
                    
                    csvwriter.writerow([csv_file, example_num, algo_name, min_bins, lb, f"{run_time:.6f}"])
                
            for algo_name, algo_func, min_bins_func in [('MB_FF', multi_bin_ff, ff),
                                                        ('MB_BF', multi_bin_bf, bf),
                                                        ('MB_WF', multi_bin_wf, wf),
                                                        ('MB_FFD', multi_bin_ffd, ffd),
                                                        ('MB_BFD', multi_bin_bfd, bfd),
                                                        ('MB_WFD', multi_bin_wfd, wfd)]:
                start_time = time.perf_counter()
                min_bins_multi = find_min_bins(weights[:], bin_capacity, algo_func, lb, min_bins_func)
                run_time = (time.perf_counter() - start_time)
                deviation = min_bins_multi - lb
                avg_results[algo_name]['total_time'] += run_time
                avg_results[algo_name]['total_deviation'] += deviation
                avg_results[algo_name]['count'] += 1
                min_bins_dict[algo_name] = {'min_bins': min_bins_multi, 'run_time': run_time}

                
                with open('bppgen_output\detail_results.csv', 'a', newline='') as csvfile:
                    csvwriter = csv.writer(csvfile)
        
                    
                    csvwriter.writerow([csv_file, example_num, algo_name, min_bins_multi, lb, f"{run_time_multi:.6f}"])
                                                                           
                 
        for algo_name, stats in avg_results.items():
            avg_time = stats['total_time'] / stats['count']
            avg_deviation = stats['total_deviation'] / stats['count']
            
            # Format the decimal to ensure that it is not displayed in scientific notation
            avg_time_formatted = "{:.10f}".format(avg_time)
            avg_deviation_formatted = "{:.10f}".format(avg_deviation)
            
            summary_writer.writerow([
                csv_file, algo_name, avg_time_formatted, avg_deviation_formatted
            ])


print(f"Summary data has been written to {summary_file}")

processing： N=100_C=1000_V1=0.01_V2=0.05.csv
processing： N=100_C=1000_V1=0.05_V2=0.6.csv
processing： N=100_C=1000_V1=0.1_V2=0.9.csv
processing： N=100_C=1000_V1=0.25_V2=0.5.csv
processing： N=100_C=1000_V1=0.4_V2=0.5.csv
processing： N=100_C=1000_V1=0.7_V2=0.9.csv
processing： N=200_C=1000_V1=0.01_V2=0.05.csv
processing： N=200_C=1000_V1=0.05_V2=0.6.csv
processing： N=200_C=1000_V1=0.1_V2=0.9.csv
processing： N=200_C=1000_V1=0.25_V2=0.5.csv
processing： N=200_C=1000_V1=0.4_V2=0.5.csv
processing： N=200_C=1000_V1=0.7_V2=0.9.csv
Summary data has been written to bppgen_output\summary_results.csv


In [ ]:
import os
import matplotlib
from adjustText import adjust_text
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.ticker import FormatStrFormatter


output_dir = 'bppgen_output'

summary_file = os.path.join(output_dir, 'summary_results.csv')

summary_data = pd.read_csv(summary_file)

algorithms = summary_data['Algorithm'].unique()
cmap = matplotlib.colormaps.get_cmap('tab10')  # Same color

csv_files = [f for f in os.listdir(output_dir) if f.startswith('N')]
output_dir = "bppgen_output\output_images"  
os.makedirs(output_dir, exist_ok=True)  # If the directory does not exist, create it


# Set Matplotlib not to use scientific notation
plt.rcParams['axes.formatter.useoffset'] = False
plt.rcParams['axes.formatter.use_locale'] = False
plt.rcParams['axes.formatter.limits'] = (-10, 10)  # Control when to switch to scientific notation

for csv_file in csv_files:
    plt.figure(figsize=(10, 5))
    file_data = summary_data[summary_data['CSV File'] == csv_file]
    texts = []


    min_time = file_data['Avg  computation time (s)'].min()
    max_time = file_data['Avg  computation time (s)'].max()
    min_deviation = file_data['Avg Deviation from LB'].min()
    max_deviation = file_data['Avg Deviation from LB'].max()

    # Dynamically set the axis
    x_margin = (max_time - min_time) * 0.05
    y_margin = (max_deviation - min_deviation) * 0.05

    
    if min_time == max_time:
        plt.xlim(left=min_time - 1, right=max_time + 1)
    else:
        plt.xlim(left=min_time - x_margin, right=max_time + x_margin)
    
    if min_deviation == max_deviation:
        plt.ylim(bottom=min_deviation - 1, top=max_deviation + 1)
    else:
        plt.ylim(bottom=min_deviation - y_margin, top=max_deviation + y_margin)

    plt.gca().xaxis.set_major_formatter(FormatStrFormatter('%.8f'))

    for i, algo in enumerate(algorithms):
        algo_data = file_data[file_data['Algorithm'] == algo]

        times = ["{:.10f}".format(x) for x in algo_data['Avg  computation time (s)']]
        deviations = ["{:.10f}".format(x) for x in algo_data['Avg Deviation from LB']]
        plt.scatter(algo_data['Avg  computation time (s)'], algo_data['Avg Deviation from LB'], 
                    label=algo, color=cmap(i % cmap.N), s=60, alpha=0.6)  
        for idx, row in algo_data.iterrows():
            text = plt.text(row['Avg  computation time (s)'], row['Avg Deviation from LB'], algo,
                            fontsize=9, ha='right')
            texts.append(text)
            
            
            plt.axhline(y=row['Avg Deviation from LB'], color='gray', linestyle='--', linewidth=0.7)
            plt.axvline(x=row['Avg  computation time (s)'], color='gray', linestyle='--', linewidth=0.7)
    
    # Whole text labels to avoid overlapping
    # adjust_text(texts, arrowprops=dict(arrowstyle='->', color='gray'))
    adjust_text(
    texts,
    only_move={'points': 'y', 'texts': 'xy'},  # move in the X and Y directions
    arrowprops=dict(arrowstyle='->', color='gray'),
    expand_points=(1.2, 1.4),  # Expand the range of points to prevent overlap
    expand_text=(1.2, 1.4),  # Extend the range of text labels
    force_text=1.0,  
    force_points=1.0  
    )


    file_base_name = os.path.splitext(csv_file)[0]
    plt.xlabel('Average computation time (s)')
    plt.ylabel('Average Deviation from LB')
    plt.title(f'Algorithm Performance for {file_base_name}')  
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()

    
    output_image_path = os.path.join(output_dir, f"{os.path.splitext(csv_file)[0]}_performance.png")
    plt.savefig(output_image_path, format='png')


    plt.show()

    plt.close()


In [ ]:
import pandas as pd


df = pd.read_csv('bppgen_output\detail_results.csv')

wf_algos = ['WF', 'WFD']
other_algos = ['FF', 'BF', 'FFD', 'BFD']
all_algos=['FF', 'BF', 'FFD', 'BFD','WF', 'WFD']
mb_algos = ['MB_FF', 'MB_BF', 'MB_WF', 'MB_FFD', 'MB_BFD', 'MB_WFD']

result_rows = []

for (file_name, example_number), group in df.groupby(['File Name', 'Example Number']):

    wf_group = group[group['Algorithm'].isin(wf_algos)]
    other_group = group[group['Algorithm'].isin(other_algos)]
    mb_group = group[group['Algorithm'].isin(mb_algos)]
    all_group=group[group['Algorithm'].isin(all_algos)]
    
    for _, wf_row in wf_group.iterrows():
        for _, other_row in other_group.iterrows():
            if wf_row['Min Bins'] <other_row['Min Bins']:
                result_rows.append({
                    'File Name': file_name,
                    'Example Number': example_number,
                    'Algorithm1': wf_row['Algorithm'],
                    'Min Bins 1': wf_row['Min Bins'],
                    'Computation Time 1 (s)': wf_row['Computation Time (s)'],
                    'Algorithm2': other_row['Algorithm'],
                    'Min Bins 2': other_row['Min Bins'],
                    'Computation Time 2 (s)': other_row['Computation Time (s)']
                })
    
    
    for _, mb_row in mb_group.iterrows():
        for _, all_algos in group.iterrows():
            
            if mb_row['Algorithm'][3:] == all_algos['Algorithm'] and mb_row['Min Bins'] < all_algos['Min Bins']:
                result_rows.append({
                    'File Name': file_name,
                    'Example Number': example_number,
                    'Algorithm1': mb_row['Algorithm'],
                    'Min Bins 1': mb_row['Min Bins'],
                    'Computation Time 1 (s)': mb_row['Computation Time (s)'],
                    'Algorithm2': all_algos['Algorithm'],
                    'Min Bins 2':all_algos['Min Bins'],
                    'Computation Time 2 (s)': all_algos['Computation Time (s)']
                })

result_df = pd.DataFrame(result_rows)

result_df.to_csv('bppgen_output/filtered_results.csv', index=False, float_format='%.6f')

print(f"Summary data has been written to {'bppgen_output/filtered_results.csv'}")